In [1]:
# Import libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [2]:
# Import datasets
df = pd.read_csv(r'C:\Users\Ahmed\Desktop\titanic\train.csv')
df_pred = pd.read_csv(r'C:\Users\Ahmed\Desktop\titanic\test.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# Impute missing data
missing_sum = df.isnull().sum().sort_values(ascending = False)
missing_percent = ((df.isnull().sum()/df.isnull().count()).sort_values(ascending = False))
missing_percent = round(missing_percent, 2)
missing_class = df.dtypes
missing_table = pd.concat([missing_sum, missing_percent, missing_class], axis =1, keys =['Total NULL', 'Percentage of Total', 'Data Type'])
missing_table.sort_values('Total NULL', ascending = False)

,Total NULL,Percentage of Total,Data Type
Cabin,687,0.77,object
Age,177,0.20,float64
Embarked,2,0.00,object
Fare,0,0.00,float64
Name,0,0.00,object
Parch,0,0.00,int64
PassengerId,0,0.00,int64
Pclass,0,0.00,int64
Sex,0,0.00,object
SibSp,0,0.00,int64


In [5]:
# Impute median for Embarked variable 
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [6]:
df['Embarked'] = df['Embarked'].fillna('S')
df['Embarked'].value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [7]:
# Impute median for Age variable
imputer_age = SimpleImputer(missing_values = np.nan, strategy = 'median')
imputer_age = imputer_age.fit(df[['Age']])
df['Age'] = imputer_age.transform(df[['Age']])

In [8]:
# We will create a new feature for it as cabin holders represent a unique passenger segment
df['Cabin'] = np.where(df['Cabin'].notnull(), 1, 0)

In [9]:
# All null values have been treated
missing_sum = df.isnull().sum().sort_values(ascending = False)
missing_percent = ((df.isnull().sum()/df.isnull().count()).sort_values(ascending = False))
missing_percent = round(missing_percent, 2)
missing_class = df.dtypes
missing_table = pd.concat([missing_sum, missing_percent, missing_class], axis =1, keys =['Total NULL', 'Percentage of Total', 'Data Type'])
missing_table.sort_values('Total NULL', ascending = False)

,Total NULL,Percentage of Total,Data Type
Age,0,0.0,float64
Cabin,0,0.0,int32
Embarked,0,0.0,object
Fare,0,0.0,float64
Name,0,0.0,object
Parch,0,0.0,int64
PassengerId,0,0.0,int64
Pclass,0,0.0,int64
Sex,0,0.0,object
SibSp,0,0.0,int64


In [10]:
# Encode relevant categorical variables (Sex, Embarked)
df['Sex'] = pd.get_dummies(df['Sex'])
Port_Embarked = pd.get_dummies(df['Embarked'], prefix = 'Embarked')
df = pd.merge(df, Port_Embarked, left_index = True, right_index = True)
df = df.drop(columns = ['Embarked', 'Embarked_C'])

In [11]:
# Passenger class is also a categorical variable for which we will create a dummy variable
Class = pd.get_dummies(df['Pclass'], prefix = 'PClass')
df = pd.merge(df, Class, left_index = True, right_index = True)
df = df.drop(columns = ['Pclass', 'PClass_3'])

In [12]:
# Passengers with a title in their name belong to a unique segment and must be isolated
df['Title'] = df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
df['Title'] = df['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 
                                   'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
df['Title'] = df['Title'].replace('Mlle', 'Miss')
df['Title'] = df['Title'].replace('Ms', 'Miss')
df['Title'] = df['Title'].replace('Mme', 'Mrs')
df['Title'] = df['Title'].map({"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5})
df['Title'] = df['Title'].fillna(0)
df.head()

,PassengerId,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_Q,Embarked_S,PClass_1,PClass_2,Title
0,1,0,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,0,0,1,0,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,1,0,0,1,0,3
2,3,1,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,0,0,1,0,0,2
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,1,0,1,1,0,3
4,5,0,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,0,0,1,0,0,1


In [13]:
# Now that we have cleaned the training data and engineered its features, we must do the same to the test data. 
# This will be done in one step as to reduce redundancy. 

imputer_agePred = SimpleImputer(missing_values = np.nan, strategy = 'median')
imputer_agePred = imputer_agePred.fit(df[['Age']])
df_pred['Age'] = imputer_agePred.transform(df_pred[['Age']])

imputer_farePred = SimpleImputer(missing_values = np.nan, strategy = 'median')
imputer_farePred = imputer_farePred.fit(df[['Fare']])
df_pred['Fare'] = imputer_farePred.transform(df_pred[['Fare']])

df_pred['Cabin'] = np.where(df_pred['Cabin'].notnull(), 1, 0)
df_pred['Sex'] = pd.get_dummies(df_pred['Sex'])
Port_Embarked_pred = pd.get_dummies(df_pred['Embarked'], prefix = 'Embarked')
df_pred = pd.merge(df_pred, Port_Embarked_pred, left_index = True, right_index = True)
df_pred = df_pred.drop(columns = ['Embarked', 'Embarked_C'])
Class_pred = pd.get_dummies(df_pred['Pclass'], prefix = 'PClass')
df_pred = pd.merge(df_pred, Class_pred, left_index = True, right_index = True)
df_pred = df_pred.drop(columns = ['Pclass', 'PClass_3'])
df_pred['Title'] = df_pred.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
df_pred['Title'] = df_pred['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 
                                   'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
df_pred['Title'] = df_pred['Title'].replace('Mlle', 'Miss')
df_pred['Title'] = df_pred['Title'].replace('Ms', 'Miss')
df_pred['Title'] = df_pred['Title'].replace('Mme', 'Mrs')
df_pred['Title'] = df_pred['Title'].map({"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5})
df_pred['Title'] = df_pred['Title'].fillna(0)
df_pred.head()

,PassengerId,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_Q,Embarked_S,PClass_1,PClass_2,Title
0,892,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,0,1,0,0,0,1
1,893,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,0,0,1,0,0,3
2,894,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,0,1,0,0,1,1
3,895,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,0,0,1,0,0,1
4,896,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,0,0,1,0,0,3


In [14]:
# Now that we have cleaned all the data and transformed all relevant features, lets define our predictors and target variables.
X = df[['Title', 'Sex', 'Age', 'Cabin', 'Fare', 'SibSp', 'Parch', 'Embarked_Q', 'Embarked_S', 'PClass_1', 'PClass_2']]
y = df[['Survived']]
print(X.shape , y.shape)

(891, 11) (891, 1)


In [15]:
# We will use a train test split of test size 1/3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [16]:
# Logistic regression model
logreg = LogisticRegression()
logreg = logreg.fit(X_train, y_train)
logreg_yTrain = logreg.predict_proba(X_train)
logreg_yTest = logreg.predict_proba(X_test)
print('Performance on train set: ', roc_auc_score(y_train, logreg_yTrain[:,1]),'Performance on test set: ', roc_auc_score(y_test, logreg_yTest[:,1]),sep="\n")

Performance on train set: 
0.8576263429204607
Performance on test set: 
0.8838571428571429


In [17]:
# Random Forest Model
from sklearn.ensemble import RandomForestClassifier
rfm = RandomForestClassifier(n_jobs = -1, n_estimators = 5000, max_depth = 4, random_state = 420)
rfm = rfm.fit(X_train, y_train)
y_pred_rfm_train = rfm.predict_proba(X_train)
y_pred_rfm_test = rfm.predict_proba(X_test)
print('Performance on train set: ', roc_auc_score(y_train, y_pred_rfm_train[:,1]),'Performance on test set: ', roc_auc_score(y_test, y_pred_rfm_test[:,1]),sep="\n")

Performance on train set: 
0.8887483740424916
Performance on test set: 
0.8869285714285713


In [18]:
# SVM Model
svc = SVC(C = 0.05, kernel = 'linear', probability = True, random_state = 42)
svc = svc.fit(X_train, y_train)
y_pred_svc_train = svc.predict_proba(X_train)
y_pred_svc_test = svc.predict_proba(X_test)
print('Performance on train set: ', roc_auc_score(y_train, y_pred_svc_train[:,1]),'Performance on test set: ', roc_auc_score(y_test, y_pred_svc_test[:,1]),sep="\n")

Performance on train set: 
0.8406561641855759
Performance on test set: 
0.8627619047619047


In [19]:
from xgboost import XGBClassifier
xgb = XGBClassifier(learning_rate = 0.0075, max_depth= 3, n_estimators= 250, random_state=42, objective= 'binary:logistic')
xgb = xgb.fit(X_train, y_train)
y_pred_xgb_train = xgb.predict_proba(X_train)
y_pred_xgb_test = xgb.predict_proba(X_test)
print('Performance on train set: ', roc_auc_score(y_train, y_pred_xgb_train[:,1]),'Performance on test set: ', roc_auc_score(y_test, y_pred_xgb_test[:,1]),sep="\n")

Performance on train set: 
0.8824553162788459
Performance on test set: 
0.8826904761904761


In [20]:
# Final prediction
pred_X = df_pred[['Title', 'Sex', 'Age', 'Cabin', 'Fare', 'SibSp', 'Parch', 'Embarked_Q', 'Embarked_S', 'PClass_1', 'PClass_2']]
pred_final = rfm.predict(pred_X)
submission_dataframe = pd.DataFrame()
submission_dataframe['PassengerId'] = df_pred['PassengerId']
submission_dataframe['Survived'] = pred_final
submission_dataframe.to_csv('titanic.csv', index = False)